In [2]:
%pip install yfinance

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: multitasking>=0.0.7 in c:\users\ruben\appdata\local\programs\python\python310\lib\site-packages (from yfinance) (0.0.10)


You should consider upgrading via the 'c:\Users\Ruben\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import numpy as np
import yfinance as yf


In [2]:
stock_symbols_ftse_mib = ["A2A.MI", "AMP.MI", "ATL.MI", "AZM.MI", "BAMI.MI", "BGN.MI", "BMED.MI", "BPE.MI", "BZU.MI", "CNHI.MI", "CPR.MI", "DIA.MI", "ENEL.MI", "ENI.MI", "EXO.MI", "FBK.MI", "G.MI", "HER.MI",
                          "IG.MI", "INW.MI", "IP.MI", "ISP.MI", "LDO.MI", "MB.MI", "MONC.MI", "NEXI.MI", "PIRC.MI", "PRY.MI", "PST.MI", "RACE.MI", "REC.MI", "SPM.MI", "SRG.MI", "STLA.MI", "STM.MI", "TEN.MI", "TIT.MI", "TRN.MI", "UCG.MI", "UNI.MI", "FTSEMIB.MI"]


In [52]:
for stock in stock_symbols_ftse_mib:
  df = yf.download(stock, start="2017-06-08", end="2022-06-08")
  df.to_csv("./Datasets/"+stock+".csv")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Calculate log returns for each stock

In [53]:
for stock in stock_symbols_ftse_mib:
  df = pd.read_csv("./Datasets/"+stock+".csv")
  df["log_ret"] = np.log(df["Close"]) - np.log(df["Close"].shift(1))
  df.to_csv("./Datasets/"+stock+".csv")

In [5]:
import statsmodels.api as sm
from statsmodels import regression

In [28]:
df = pd.read_csv("./Datasets/"+"A2A.MI"+".csv")
df_ftsemib = pd.read_csv("./Datasets/"+"FTSEMIB.MI"+".csv")
df = df.dropna()
df_ftsemib = df_ftsemib.dropna()
rolling_number = 0
df_ftsemib = df_ftsemib[rolling_number: rolling_number+180]
df = df[rolling_number: rolling_number+180]


In [29]:
df.shape

(180, 9)

In [30]:
df_ftsemib.shape

(180, 9)

In [36]:
y = df["log_ret"]
X = df_ftsemib["log_ret"]
X = sm.add_constant(X)
model = regression.linear_model.OLS(y, X).fit()
print(model.summary())
print("Alpha: %s - Beta: %s" % (model.params[0], model.params[1]))


                            OLS Regression Results                            
Dep. Variable:                log_ret   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                  0.290
Method:                 Least Squares   F-statistic:                     73.97
Date:                Wed, 08 Jun 2022   Prob (F-statistic):           4.01e-15
Time:                        17:50:09   Log-Likelihood:                 572.49
No. Observations:                 180   AIC:                            -1141.
Df Residuals:                     178   BIC:                            -1135.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0007      0.001     -0.944      0.3

In [5]:
from tqdm import tqdm
import os


In [60]:
df_ftsemib = pd.read_csv("./Datasets/"+"FTSEMIB.MI"+".csv")
df_ftsemib = df_ftsemib.dropna()
for stock in tqdm(stock_symbols_ftse_mib):
  df = pd.read_csv("./Datasets/"+stock+".csv")
  df = df.dropna()
  time_index = 0
  for time_index in range(0, len(df_ftsemib), 7):
    if time_index + 180 < df.shape[0]:
      time_window_stock = df[time_index: time_index+180]
      time_window_ftsemib = df_ftsemib[time_index: time_index+180]
      X = time_window_ftsemib["log_ret"]
      y = time_window_stock["log_ret"]
      X = sm.add_constant(X)
      model = regression.linear_model.OLS(y, X).fit()
      if not os.path.exists("./Models/"+stock):
        os.makedirs("./Models/"+stock)
      model.save("./Models/"+stock+"/"+str(time_index)+".pickle")
    
    

100%|██████████| 41/41 [00:26<00:00,  1.55it/s]


Extract from each model the parameters

In [3]:
dataframe_columns = ["Stock Symbol","Week Number","Alpha","Beta","R-Squared","Res Std. Error", "Stock Weekly Return"]

In [7]:
# from statsmodels.iolib.smpickle import load_pickle
results_df = pd.DataFrame(columns=dataframe_columns)
df_ftsemib = pd.read_csv("./Datasets/"+"FTSEMIB.MI"+".csv")
for sym in stock_symbols_ftse_mib:
  i=0
  current_stock_dataset = pd.read_csv("./Datasets/"+sym+".csv")
  for file in os.listdir("./Models/"+sym):
    model = sm.load("./Models/"+sym+"/"+file)
    last_week_ret = current_stock_dataset.iloc[180+(i-7)]["log_ret"]
    current_week_ret = current_stock_dataset.iloc[180+i]["log_ret"]
    weekly_return = current_week_ret-last_week_ret
    results_df.loc[len(results_df)] = [sym, i, model.params[0],
                                       model.params[1], model.rsquared,  model.ess, weekly_return]
    i+=7
results_df.to_csv("./results.csv")


In [10]:
df_ftsemib.iloc[181+i]["log_ret"]


-0.008378265645998